<h3 style="text-align: left; color: red;">Librerías importadas</h3>

A continuación, se describe el uso de cada librería en nuestro proyecto:
- pandas y numpy: Se usan para el manejo óptimo de los archivos "csv", ya que manejarlos en su mismo formato lo vuelve complejo de forma innecesaria.
- matplotlib y seaborn: Se usan para la obtencion de diagramas y gráficas de los datos.
- sklearn: Se usó esta librería contiene todos los métodos de imputación a usarse previo al modelado.
- os, shutil y glob: Se usaron estas librerías para el correcto manejo y organización de los archivos obtenidos.
- Parallel y delayed: Se usaron estas librerías para el procesamiento en paralelo de los archivos csv, ya que procesarlos de forma secuencial requiere una gran cantidad de recursos.

In [149]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
import os
import shutil
import glob

<h3 style="text-align: left; color: red;">Lectura de csv's</h3>

Leemos los archivos que se encuentran en la carpeta desde donde se ejecuta el código y los movemos a la carpeta "Archivos No Imputados"

In [150]:
ruta_archivos_csv_origen = os.getcwd() # Obtenemos la ruta desde donde se ejecuta el Notebook
ruta_archivos_csv_destino = os.path.join(ruta_archivos_csv_origen, "Archivos No Imputados") # Definimos la ruta hacia donde van los archivos csv despues de leerse
os.makedirs("Archivos No Imputados", exist_ok = True) # Creamos la carpeta si no existe


def leer_mover_archivos (nombre_archivo_csv):
    ruta_origen_archivos_csv = os.path.join(ruta_archivos_csv_origen, nombre_archivo_csv) # Definimos la ruta actual del archivo csv
    ruta_carpeta_archivos_csv = os.path.join(ruta_archivos_csv_destino, nombre_archivo_csv) # Definimos la ruta donde deberia estar el archivo tras ejecutarse el codigo

    # Si el archivo se encuentra en la ruta de origen (desde donde se ejecuta el notebook)
    if os.path.exists(ruta_origen_archivos_csv): 

        archivo_pandas = pd.read_csv(ruta_origen_archivos_csv, low_memory = False) # Leemos el archivo csv
        archivo_pandas = archivo_pandas.reset_index(drop = True) 
        shutil.move(ruta_origen_archivos_csv, ruta_carpeta_archivos_csv) # Movemos el archivo csv desde la ruta de origen hacia la carpeta Archivos No Imputados
        print(f"{nombre_archivo_csv} leido y movido a la carpeta Archivos No Imputados")
        return archivo_pandas
    
    # Si el archivo se encuentra dentro de la carpeta "Archivos No Imputados"
    elif os.path.exists(ruta_carpeta_archivos_csv):

        archivo_pandas = pd.read_csv(ruta_carpeta_archivos_csv, low_memory = False) # Leemos el archivo csv
        archivo_pandas = archivo_pandas.reset_index(drop = True)
        print(f"{nombre_archivo_csv} ya se encuentra en Archivos No Imputados. Leyendo de nuevo")
        return archivo_pandas
    
    # Si el archivo no se encuentra en ninguna carpeta accesible por nuestro codigo
    else: 

        print(f"No se encontro el archivo {nombre_archivo_csv}")
        return None
       
ciudad_dia    = leer_mover_archivos("city_day.csv")
ciudad_hora   = leer_mover_archivos("city_hour.csv")
estacion_dia  = leer_mover_archivos("station_day.csv")
estacion_hora = leer_mover_archivos("station_hour.csv")
estaciones    = leer_mover_archivos("stations.csv")


# Creamos un diccionario con los nombres de los archivos csv y el nombre con el que se guardaran los archivos 
# lista = {
#     "ciudad_dia":"city_day.csv",
#     "ciudad_hora":"city_hour.csv",
#     "estacion_dia":"station_day.csv",
#     "estacion_hora":"station_hour.csv",
#     "estaciones":"stations.csv"
#     }

# class FicheroArchivos:
#     listaDataFrames = []
#     def __init__(self, listaCSV):
        
#         for x,y in listaCSV.items():
#             self.listaDataFrames.append(leer_mover_archivos(y)) 
# a = FicheroArchivos(lista)
# a.listaDataFrames[0]

city_day.csv leido y movido a la carpeta Archivos No Imputados
city_hour.csv leido y movido a la carpeta Archivos No Imputados
station_day.csv leido y movido a la carpeta Archivos No Imputados
station_hour.csv leido y movido a la carpeta Archivos No Imputados
stations.csv leido y movido a la carpeta Archivos No Imputados


Para la obtener más claridad acerca de las columnas y el tipo de datos, usamos describe() e info() para filtrar lo que necesitamos usar.

<h3 style="text-align: left; color: red;">Datos por Ciudad-día y filtrado</h3>

In [151]:
ciudad_dia.describe()
ciudad_dia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29531 entries, 0 to 29530
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        29531 non-null  object 
 1   Date        29531 non-null  object 
 2   PM2.5       24933 non-null  float64
 3   PM10        18391 non-null  float64
 4   NO          25949 non-null  float64
 5   NO2         25946 non-null  float64
 6   NOx         25346 non-null  float64
 7   NH3         19203 non-null  float64
 8   CO          27472 non-null  float64
 9   SO2         25677 non-null  float64
 10  O3          25509 non-null  float64
 11  Benzene     23908 non-null  float64
 12  Toluene     21490 non-null  float64
 13  Xylene      11422 non-null  float64
 14  AQI         24850 non-null  float64
 15  AQI_Bucket  24850 non-null  object 
dtypes: float64(13), object(3)
memory usage: 3.6+ MB


In [152]:
# Descartamos las variables que no serán usadas, dejando las string para su posterior union
ciudad_dia_filtrado_string = ciudad_dia[['City', 'Date', 'PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3', 'AQI']]
ciudad_dia_filtrado_string.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29531 entries, 0 to 29530
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   City    29531 non-null  object 
 1   Date    29531 non-null  object 
 2   PM2.5   24933 non-null  float64
 3   PM10    18391 non-null  float64
 4   NO2     25946 non-null  float64
 5   CO      27472 non-null  float64
 6   SO2     25677 non-null  float64
 7   O3      25509 non-null  float64
 8   AQI     24850 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.0+ MB


<h3 style="text-align: left; color: red;">Datos por Ciudad-hora y filtrado</h3>

In [153]:
ciudad_hora.describe()
ciudad_hora.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707875 entries, 0 to 707874
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   City        707875 non-null  object 
 1   Datetime    707875 non-null  object 
 2   PM2.5       562787 non-null  float64
 3   PM10        411138 non-null  float64
 4   NO          591243 non-null  float64
 5   NO2         590753 non-null  float64
 6   NOx         584651 non-null  float64
 7   NH3         435333 non-null  float64
 8   CO          621358 non-null  float64
 9   SO2         577502 non-null  float64
 10  O3          578667 non-null  float64
 11  Benzene     544229 non-null  float64
 12  Toluene     487268 non-null  float64
 13  Xylene      252046 non-null  float64
 14  AQI         578795 non-null  float64
 15  AQI_Bucket  578795 non-null  object 
dtypes: float64(13), object(3)
memory usage: 86.4+ MB


In [154]:
# Descartamos las variables que no serán usadas, dejando las string para su posterior union
ciudad_hora_filtrado_string = ciudad_hora[['City', 'Datetime', 'PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3', 'AQI']]
ciudad_hora_filtrado_string.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707875 entries, 0 to 707874
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   City      707875 non-null  object 
 1   Datetime  707875 non-null  object 
 2   PM2.5     562787 non-null  float64
 3   PM10      411138 non-null  float64
 4   NO2       590753 non-null  float64
 5   CO        621358 non-null  float64
 6   SO2       577502 non-null  float64
 7   O3        578667 non-null  float64
 8   AQI       578795 non-null  float64
dtypes: float64(7), object(2)
memory usage: 48.6+ MB


<h3 style="text-align: left; color: red;">Datos por Estación-día y filtrado</h3>


In [155]:
estacion_dia.describe()
estacion_dia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108035 entries, 0 to 108034
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   StationId   108035 non-null  object 
 1   Date        108035 non-null  object 
 2   PM2.5       86410 non-null   float64
 3   PM10        65329 non-null   float64
 4   NO          90929 non-null   float64
 5   NO2         91488 non-null   float64
 6   NOx         92535 non-null   float64
 7   NH3         59930 non-null   float64
 8   CO          95037 non-null   float64
 9   SO2         82831 non-null   float64
 10  O3          82467 non-null   float64
 11  Benzene     76580 non-null   float64
 12  Toluene     69333 non-null   float64
 13  Xylene      22898 non-null   float64
 14  AQI         87025 non-null   float64
 15  AQI_Bucket  87025 non-null   object 
dtypes: float64(13), object(3)
memory usage: 13.2+ MB


In [156]:
# Descartamos las variables que no serán usadas, dejando las string para su posterior union
estacion_dia_filtrado_string = estacion_dia[['Date', 'PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3', 'AQI']]
estacion_dia_filtrado_string.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108035 entries, 0 to 108034
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Date    108035 non-null  object 
 1   PM2.5   86410 non-null   float64
 2   PM10    65329 non-null   float64
 3   NO2     91488 non-null   float64
 4   CO      95037 non-null   float64
 5   SO2     82831 non-null   float64
 6   O3      82467 non-null   float64
 7   AQI     87025 non-null   float64
dtypes: float64(7), object(1)
memory usage: 6.6+ MB


<h3 style="text-align: left; color: red;">Datos por Estación-hora y filtrado</h3>


In [157]:
estacion_hora.describe()
estacion_hora.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2589083 entries, 0 to 2589082
Data columns (total 16 columns):
 #   Column      Dtype  
---  ------      -----  
 0   StationId   object 
 1   Datetime    object 
 2   PM2.5       float64
 3   PM10        float64
 4   NO          float64
 5   NO2         float64
 6   NOx         float64
 7   NH3         float64
 8   CO          float64
 9   SO2         float64
 10  O3          float64
 11  Benzene     float64
 12  Toluene     float64
 13  Xylene      float64
 14  AQI         float64
 15  AQI_Bucket  object 
dtypes: float64(13), object(3)
memory usage: 316.1+ MB


In [158]:
# Descartamos las variables que no serán usadas, dejando las string para su posterior union
estacion_hora_filtrado_string = estacion_hora[['Datetime', 'PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3', 'AQI']]
estacion_hora_filtrado_string.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2589083 entries, 0 to 2589082
Data columns (total 8 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Datetime  object 
 1   PM2.5     float64
 2   PM10      float64
 3   NO2       float64
 4   CO        float64
 5   SO2       float64
 6   O3        float64
 7   AQI       float64
dtypes: float64(7), object(1)
memory usage: 158.0+ MB


<h3 style="text-align: left; color: red;">Datos de Estaciones</h3>

Este archivo al solo contener un id y campos de tipo objeto/string, no será usado directamente para la imputación. Tiene una función importante en la unión de "tablas" por su campo "StationId"

In [159]:
estaciones.describe()
estaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   StationId    230 non-null    object
 1   StationName  230 non-null    object
 2   City         230 non-null    object
 3   State        230 non-null    object
 4   Status       133 non-null    object
dtypes: object(5)
memory usage: 9.1+ KB


<h3 style="text-align: left; color: red;">Función para contar los valores NaN totales</h3>

In [160]:
# Definimos una funcion para verificar los valores NaN que tenemos en cada fila, si al menos existe 1 valor NaN en la fila, toda la fila se declara como tal
def contar_filas_con_nan_en_columnas_especificas(nombre_archivo_csv):
    archivo = pd.read_csv(nombre_archivo_csv)
    
    variables = ['PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3', 'AQI']
    archivo_filtrado = archivo[variables]
    
    filas_con_nan = archivo_filtrado.isna().any(axis = 1) # axis = 1 ya que revisamos por fila, operando a lo largo de las columnas
    cantidad_filas_con_nan = filas_con_nan.sum() # Sumamos las filas que contienen al menos un campo en NaN
    
    return cantidad_filas_con_nan

print("Valores NaN del csv ciudad-hora: ",contar_filas_con_nan_en_columnas_especificas("Archivos no imputados/city_hour.csv"))
print("Valores NaN del csv ciudad-dia: ",contar_filas_con_nan_en_columnas_especificas("Archivos no imputados/city_day.csv"))
print("Valores NaN del csv estacion-hora: ",contar_filas_con_nan_en_columnas_especificas("Archivos no imputados/station_hour.csv"))
print("Valores NaN del csv estacion-dia: ",contar_filas_con_nan_en_columnas_especificas("Archivos no imputados/station_day.csv"))

Valores NaN del csv ciudad-hora:  366370
Valores NaN del csv ciudad-dia:  13521


C:\Users\david\AppData\Local\Temp\ipykernel_13804\2595521705.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  archivo = pd.read_csv(nombre_archivo_csv)


Valores NaN del csv estacion-hora:  1554807
Valores NaN del csv estacion-dia:  58038


<h4 style="text-align: left; color: red;">Imputación mediante SimpleImputer, KNN y PCA en paralelo</h4>

A continuación, definimos las funciones que conforman la función principal para imputar nuestros valores vacíos mediante un barrido rápido con SimpleImputer y un posterior complemento con KNN y PCA. Esto en división por "chunks" o bloques para ejecutar los procesos en paralelo y optimizar los recursos limitados del dispositivo. 

<h5 style="text-align: left; color: red;">Función de barrido inicial con SimpleImputer</h5>

Para que el barrido sea rápido, se utilizo el parámetro "mean" de SimpleImputer, que toma la media de la columna para reemplazar los valores faltantes. Todo esto preservando las columnas e índices del archivo original.